In [46]:
# Install dependencies if needed (uncomment to run)
!pip install faiss-cpu transformers torch pandas numpy
!pip install faiss-cpu

import sys
print(sys.executable)

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Python314\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
c:\Users\ArisK\AppData\Local\Programs\Python\Python310\python.exe



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Python314\python.exe -m pip install --upgrade pip


In [47]:
import pandas as pd
import numpy as np
import torch
import faiss
import os
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

In [48]:
import os

BASE_PATH = input("Δώσε το path του φακέλου IR2025 (π.χ. C:/IR2025): ").strip()
OUTPUT_DIR = os.path.join(BASE_PATH, "outputs")
os.makedirs(OUTPUT_DIR, exist_ok=True)

if not os.path.exists(BASE_PATH):
    raise FileNotFoundError(f"Ο φάκελος δεν υπάρχει: {BASE_PATH}")

# Load Documents
docs_path = os.path.join(BASE_PATH, "documents.csv")
documents_df = pd.read_csv(docs_path)
print(f"Loaded {len(documents_df)} documents.")
print(documents_df.head())

# Load Queries
queries_path = os.path.join(BASE_PATH, "queries.csv")
queries_df = pd.read_csv(queries_path)
print(f"Loaded {len(queries_df)} queries.")
print(queries_df.head())

Loaded 18316 documents.
       ID                                               Text
0  193157  Support towards the Europe PMC initiative-Cont...
1  193158  Support to the Vice-Presidents of the ERC Scie...
2  193159  Implementation of activities described in the ...
3  193160  Monitoring Atmospheric Composition and Climate...
4  193161  Pre-Operational Marine Service Continuity in T...
Loaded 10 queries.
    ID                                               Text
0  Q01  EUTRAVEL Optimodal European Travel Ecosystem E...
1  Q02  Track And Know Big Data for Mobility Tracking ...
2  Q03  SELIS, Towards a Shared European Logistics Int...
3  Q04  TYPHON Polyglot and Hybrid Persistence Archite...
4  Q05  CHARIOT Cognitive Heterogeneous Architecture f...


## 1+2. Προεπεξεργασία και Δημιουργία Embeddings
Χρήση του `all-MiniLM-L6-v2`.

Η συνάρτηση `encode_texts` με τη βοήθεια της `transformers` βιβλιοθήκης παίρνει τα κείμενα και δημιουργεί embeddings για τα δεδομένα μας. Μέσα στην συνάρτηση πραγματοποιούνται: 

1. **Batching**: Χωρίζουμε τα δεδομένα σε μικρά πακέτα (π.χ. 32 κείμενα τη φορά) για να μην υπερφορτώσουμε τη μνήμη.
2. **Model Inference (Forward Pass)**: Περνάμε τα tokens από το μοντέλο Transformer και παίρνουμε ως έξοδο ένα διάνυσμα για κάθε token της πρότασης.
3. **Mean Pooling**: Υπολογίζουμε τον μέσο όρο των διανυσμάτων όλων των tokens μιας πρότασης (αγνοώντας τα padding tokens), ώστε να καταλήξουμε σε ένα μοναδικό διάνυσμα (embedding) που αντιπροσωπεύει ολόκληρη την πρόταση.
4. **Normalization**: Κανονικοποιούμε το τελικό διάνυσμα (L2 norm) ώστε να έχει μήκος 1. Αυτό γίνεται γιατί η ομοιότητα συνημιτόνου (Cosine Similarity) ισοδυναμεί με το εσωτερικό γινόμενο (Dot Product) όταν τα διανύσματα είναι κανονικοποιημένα.

#### Προεπεξεργασία των δεδομένων:
1. **Καθαρισμός (NaN handling)**: Μετατροπή κενών τιμών (NaN) σε κενά strings.
2. **Tokenization**: Χωρισμός του κειμένου σε tokens (λέξεις ή τμήματα λέξεων).
3. **Mapping**: Αντιστοίχιση των tokens σε μοναδικούς αριθμούς (IDs).
4. **Special Tokens**: Προσθήκη ειδικών συμβόλων αρχής [CLS] και τέλους [SEP].
5. **Padding**: Προσθήκη "κενών" tokens ώστε όλες οι προτάσεις στο πακέτο να έχουν το ίδιο μήκος.
6. **Truncation**: Αποκοπή του κειμένου που ξεπερνά το όριο (128 tokens) για να χωράει στη μνήμη.
7. **Tensors**: Μετατροπή των δεδομένων σε μορφή PyTorch Tensors για είσοδο στο μοντέλο.

In [49]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def encode_texts(texts, batch_size=32):
    model.eval()
    all_embeddings = []
    
    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size), desc="Encoding"):
            batch_texts = texts[i:i+batch_size]
            # Προεπεξεργασία των δεδομένων
            inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt', max_length=128)
            # Υπολογισμός embeddings
            outputs = model(**inputs)
            # Mean Pooling - Λήψη του attention mask υπόψη για σωστό μέσο όρο
            token_embeddings = outputs.last_hidden_state
            attention_mask = inputs['attention_mask']
            
            input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
            sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
            sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
            params = sum_embeddings / sum_mask
            
            # Κανονικοποίηση embeddings για cosine similarity
            params = torch.nn.functional.normalize(params, p=2, dim=1)
            all_embeddings.append(params.numpy())
            
    return np.vstack(all_embeddings)

In [50]:

embedding_file = os.path.join(OUTPUT_DIR, "document_embeddings.npy")

if os.path.exists(embedding_file):
    print("Loading existing embeddings...")
    doc_embeddings = np.load(embedding_file)
else:
    print("Generating embeddings...")
    docs_text = documents_df["Text"].fillna("").tolist()
    doc_embeddings = encode_texts(docs_text)
    np.save(embedding_file, doc_embeddings)

    
print(f"Embeddings shape: {doc_embeddings.shape}")

Generating embeddings...


Encoding: 100%|██████████| 573/573 [03:49<00:00,  2.49it/s]


Embeddings shape: (18316, 384)


## 3. Indexing με FAISS
Χρήση `IndexFlatIP` για Cosine Similarity αφού έχουμε κανονικοποιημένα δεδομένα.

In [51]:
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)

index.add(doc_embeddings)
print(f"Total vectors in index: {index.ntotal}")

Total vectors in index: 18316


## 4. Δημιουργία embeddings για queries και αναζήτηση documents για κάθε query

In [52]:
# Δημιουργία embeddings για queries
queries_text = queries_df['Text'].fillna("").tolist()
print("Encoding queries...")
query_embeddings = encode_texts(queries_text, batch_size=32)



Encoding queries...


Encoding: 100%|██████████| 1/1 [00:00<00:00,  8.29it/s]


## 5. Trec_eval

Για να ετοιμάσουμε τα δεδομένα για το trec_eval:

1. Διασχίσαμε τα αποτελέσματα του FAISS για κάθε ερώτημα.
2. Αντιστοιχίσαμε τους αύξοντες αριθμούς (indices) του FAISS με τα πραγματικά IDs ερωτημάτων και εγγράφων από τα αρχικά CSV αρχεία.
3. Μορφοποιήσαμε κάθε αποτέλεσμα σε μια γραμμή κειμένου με τη δομή: QueryID Q0 DocID Rank Score RunName.

In [53]:
run_name = "Phase2_Transformer_FAISS"

k_values = [20, 30, 50]


for k in k_values:
    print(f"Searching top-{k} docs per query...")
    D, I = index.search(query_embeddings, k)

    results_lines = []
    for q_idx, neighbors in enumerate(I):
        query_id = queries_df.iloc[q_idx]["ID"]

        for rank, doc_idx in enumerate(neighbors):
            doc_id = documents_df.iloc[doc_idx]["ID"]
            score = float(D[q_idx][rank])  # cosine similarity

            # TREC run format: qid Q0 docid rank score runname
            results_lines.append(
                f"{query_id} Q0 {doc_id} {rank+1} {score:.6f} {run_name}"
            )

    output_file = os.path.join(OUTPUT_DIR, f"results_phase2_k{k}.txt")
    with open(output_file, "w", encoding="utf-8") as f:
        f.write("\n".join(results_lines))

    print(f"Saved: {output_file} (lines={len(results_lines)})")

Searching top-20 docs per query...
Saved: C:\Users\ArisK\Desktop\IR20252026\outputs\results_phase2_k20.txt (lines=200)
Searching top-30 docs per query...
Saved: C:\Users\ArisK\Desktop\IR20252026\outputs\results_phase2_k30.txt (lines=300)
Searching top-50 docs per query...
Saved: C:\Users\ArisK\Desktop\IR20252026\outputs\results_phase2_k50.txt (lines=500)


## 6. Evaluation

In [54]:
import subprocess
import pandas as pd
import re
from pathlib import Path

pd.options.display.float_format = "{:.5f}".format

# =========================
# 1. INPUT PATHS
# =========================
base_path_input = input("Δώσε το path του φακέλου IR2025: ").strip().strip('"').strip("'")
BASE_PATH = Path(base_path_input)

if not BASE_PATH.exists():
    raise FileNotFoundError(f"Δεν βρέθηκε: {BASE_PATH}")

treceval_input = input("Δώσε το path του trec_eval.exe: ").strip().strip('"').strip("'")
TREC_EVAL_EXE = Path(treceval_input)

if not TREC_EVAL_EXE.exists():
    raise FileNotFoundError(f"Δεν βρέθηκε: {TREC_EVAL_EXE}")

OUT_DIR = BASE_PATH / "outputs"
QRELS_TXT = BASE_PATH / "qrels.txt"

if not OUT_DIR.exists():
    raise FileNotFoundError(f"Δεν βρέθηκε outputs folder: {OUT_DIR}")
if not QRELS_TXT.exists():
    raise FileNotFoundError(f"Δεν βρέθηκε qrels.txt")

# =========================
# 2. RUN FILES (Phase 2)
# =========================
RUN_FILES = [
    OUT_DIR / "results_phase2_k20.txt",
    OUT_DIR / "results_phase2_k30.txt",
    OUT_DIR / "results_phase2_k50.txt",
]

KS = [5, 10, 15, 20]

# =========================
# 3. TREC_EVAL HELPERS
# =========================
def run_trec_eval_for_k(trec_eval_exe, qrels, run_path, k):
    """
    Τρέχει trec_eval για map + P@k (ΜΟΝΟ ένα k κάθε φορά)
    """
    cmd = [
        str(trec_eval_exe),
        "-q",
        "-m", "map",
        "-m", f"P.{k}",
        str(qrels),
        str(run_path),
    ]
    result = subprocess.run(cmd, capture_output=True, text=True, check=True)
    return result.stdout


def parse_output_to_df(output: str) -> pd.DataFrame:
    rows = []
    for line in output.strip().splitlines():
        parts = re.split(r"\s+", line.strip())
        if len(parts) != 3:
            continue

        metric, qid, val = parts

        m = re.search(r"^P[._]?(\d+)$", metric)
        if m:
            metric = f"P@{m.group(1)}"

        rows.append({
            "Query": qid.strip().replace("\ufeff", ""),
            "Metric": metric,
            "Value": float(val)
        })

    return pd.DataFrame(rows)

# =========================
# 4. PHASE 2 EVALUATION
# =========================
summary_rows = []

for run_path in RUN_FILES:
    if not run_path.exists():
        print(f"⚠️ Skip (δεν βρέθηκε): {run_path.name}")
        continue

    print(f"\n=== Phase2 αξιολόγηση για: {run_path.name} ===")

    combined_df = pd.DataFrame()

    for k in KS:
        out = run_trec_eval_for_k(TREC_EVAL_EXE, QRELS_TXT, run_path, k)
        df_k = parse_output_to_df(out)
        pivot_k = df_k.pivot(index="Query", columns="Metric", values="Value")
        combined_df = pd.concat([combined_df, pivot_k], axis=1)

    # Αφαίρεση duplicate map columns
    combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]

    ordered_cols = ["map"] + [f"P@{k}" for k in KS]
    final = combined_df.reindex(columns=ordered_cols)

    display(final)

    per_query_csv = OUT_DIR / f"{run_path.stem}_per_query.csv"
    final.to_csv(per_query_csv)
    print("💾 Saved:", per_query_csv)

    if "all" in final.index:
        row = final.loc["all"].to_dict()
        row["run"] = run_path.name
        summary_rows.append(row)

summary_df_phase2 = pd.DataFrame(summary_rows).set_index("run")
summary_df_phase2 = summary_df_phase2.reindex(columns=["map", "P@5", "P@10", "P@15", "P@20"])

print("\n=== Phase2 ΣΥΓΚΕΝΤΡΩΤΙΚΑ (all) ===")
display(summary_df_phase2)

summary_csv = OUT_DIR / "phase2_trec_eval_summary_all.csv"
summary_df_phase2.to_csv(summary_csv)
print("💾 Saved:", summary_csv)

# =========================
# 5. PHASE 1 vs PHASE 2
# =========================
phase1_csv = OUT_DIR / "phase1_trec_eval_summary_all.csv"
phase2_csv = OUT_DIR / "phase2_trec_eval_summary_all.csv"

df1 = pd.read_csv(phase1_csv, index_col=0)
df2 = pd.read_csv(phase2_csv, index_col=0)

cols = ["map", "P@5", "P@10", "P@15", "P@20"]
df1 = df1[cols]
df2 = df2[cols]

df1.index = ["Phase1_" + idx for idx in df1.index]
df2.index = ["Phase2_" + idx for idx in df2.index]

comparison = pd.concat([df1, df2], axis=0)

print("\n=== ΣΥΓΚΡΙΣΗ Phase1 vs Phase2 ===")
display(comparison)

comparison_csv = OUT_DIR / "comparison_phase1_vs_phase2.csv"
comparison.to_csv(comparison_csv)
print("💾 Saved:", comparison_csv)

print("\nDONE — Phase2 evaluation ολοκληρώθηκε ✔️")



=== Phase2 αξιολόγηση για: results_phase2_k20.txt ===


Metric,map,P@5,P@10,P@15,P@20
Query,,,,,
Q01,0.03330,0.20000,0.20000,0.13330,0.10000
Q02,0.09850,0.20000,0.10000,0.13330,0.10000
Q03,0.27990,0.60000,0.50000,0.33330,0.30000
Q04,0.22390,0.40000,0.40000,0.33330,0.30000
Q05,0.15520,0.40000,0.30000,0.26670,0.25000
Q06,0.30450,0.80000,0.40000,0.40000,0.40000
Q07,0.21160,0.40000,0.40000,0.40000,0.35000
Q08,0.15250,0.40000,0.30000,0.20000,0.20000
Q09,0.32560,0.40000,0.50000,0.53330,0.60000


💾 Saved: C:\Users\ArisK\Desktop\IR20252026\outputs\results_phase2_k20_per_query.csv

=== Phase2 αξιολόγηση για: results_phase2_k30.txt ===


Metric,map,P@5,P@10,P@15,P@20
Query,,,,,
Q01,0.04150,0.20000,0.20000,0.13330,0.10000
Q02,0.12150,0.20000,0.10000,0.13330,0.10000
Q03,0.27990,0.60000,0.50000,0.33330,0.30000
Q04,0.24570,0.40000,0.40000,0.33330,0.30000
Q05,0.18540,0.40000,0.30000,0.26670,0.25000
Q06,0.36530,0.80000,0.40000,0.40000,0.40000
Q07,0.23240,0.40000,0.40000,0.40000,0.35000
Q08,0.18380,0.40000,0.30000,0.20000,0.20000
Q09,0.37970,0.40000,0.50000,0.53330,0.60000


💾 Saved: C:\Users\ArisK\Desktop\IR20252026\outputs\results_phase2_k30_per_query.csv

=== Phase2 αξιολόγηση για: results_phase2_k50.txt ===


...

💾 Saved: C:\Users\ArisK\Desktop\IR20252026\outputs\results_phase2_k50_per_query.csv

=== Phase2 ΣΥΓΚΕΝΤΡΩΤΙΚΑ (all) ===


,map,P@5,P@10,P@15,P@20
run,,,,,
results_phase2_k20.txt,0.20390,0.42000,0.34000,0.29330,0.28000
results_phase2_k30.txt,0.23280,0.42000,0.34000,0.29330,0.28000
results_phase2_k50.txt,0.25560,0.42000,0.34000,0.29330,0.28000


💾 Saved: C:\Users\ArisK\Desktop\IR20252026\outputs\phase2_trec_eval_summary_all.csv

=== ΣΥΓΚΡΙΣΗ Phase1 vs Phase2 ===


,map,P@5,P@10,P@15,P@20
Phase1_run_bm25_k20.txt,0.54630,0.82000,0.67000,0.58000,0.52500
Phase1_run_bm25_k30.txt,0.60110,0.82000,0.67000,0.58000,0.52500
Phase1_run_bm25_k50.txt,0.63850,0.82000,0.67000,0.58000,0.52500
Phase2_results_phase2_k20.txt,0.20390,0.42000,0.34000,0.29330,0.28000
Phase2_results_phase2_k30.txt,0.23280,0.42000,0.34000,0.29330,0.28000
Phase2_results_phase2_k50.txt,0.25560,0.42000,0.34000,0.29330,0.28000


💾 Saved: C:\Users\ArisK\Desktop\IR20252026\outputs\comparison_phase1_vs_phase2.csv

DONE — Phase2 evaluation ολοκληρώθηκε ✔️
